In [3]:

'''
MIT License

Copyright (c) 2023 Fast Data Science Ltd (https://fastdatascience.com)

Maintainer: Thomas Wood

Tutorial at https://fastdatascience.com/drug-named-entity-recognition-python-library/

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''

import csv
import datetime
import os
import subprocess
import xml.sax
from sys import platform

# Example URL of MeSH dump: https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/desc2023.xml

mesh_xml_file_name = f"desc{datetime.datetime.now().year}.xml"
url = f"https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/{mesh_xml_file_name}"

'''
if os.path.exists(mesh_xml_file_name):
    print(f"Removing old XML file {mesh_xml_file_name}.")
    os.remove(mesh_xml_file_name)

print(
    f"Downloading MeSH XML dump from {url}. If this URL doesn't work, please navigate to https://www.nlm.nih.gov/ and search the site for a MeSH data dump in XML format.")

print(f"Platform is {platform}.")
if "win" in platform:  # if we are on Windows, use curl.exe (supported in Windows 10 and up)
    wget = subprocess.Popen(["curl.exe", "--output", mesh_xml_file_name, "--url", url])
else:
    wget = subprocess.Popen(["wget", url])

os.waitpid(wget.pid, 0)
'''

print(f"Downloaded MeSH XML dump from {url}.")

IMPORTANT_TAGS = {'DescriptorName', 'String', 'DescriptorUI', 'DescriptorRecord', 'TreeNumber', 'Term'}


# define a Custom ContentHandler class that extends ContenHandler
class CustomContentHandler(xml.sax.ContentHandler):
    def __init__(self, writer):
        self.writer = writer
        self.writer.writerow(["Mesh ID", "Generic name", "Common name", "Synonyms"])
        self.postCount = 0
        self.entryCount = 0
        self.is_in = dict([n, False] for n in IMPORTANT_TAGS)
        self.title = ""
        self.id = ""
        self.tree_numbers = set()
        self.terms = set()
        self.generic_names = set()
        self.path = []
        self.RecordPreferredTermYN = ""

    # Handle startElement
    def startElement(self, tagName, attrs):
        self.path.append(tagName)
        if tagName == 'Term':
            if "RecordPreferredTermYN" in attrs.getNames():
                self.RecordPreferredTermYN = attrs.getValue("RecordPreferredTermYN")
        if tagName in IMPORTANT_TAGS:
            self.is_in[tagName] = True

    # Handle endElement
    def endElement(self, tagName):
        self.path = self.path[:-1]
        if tagName == "Term":
            self.RecordPreferredTermYN = ""
        if tagName == "DescriptorRecord":
            # if True or self.title.upper() in drugs_finder.drug_variant_to_canonical:
            is_include = False
            for t in self.tree_numbers:
                if t.startswith("C"):
                    is_include = True
                else:
                    is_include = False
                    break
                # if len(t.split('.')) < 4:
                #     is_include = False
                #     break
            if is_include:
                self.writer.writerow([self.id, "|".join(self.generic_names), self.title, "|".join(self.terms), "|".join(self.tree_numbers)])
                print(self.id, self.title, self.tree_numbers, self.terms)
            self.title = ""
            self.id = ""
            self.tree_numbers = set()
            self.terms = set()
            self.generic_names = set()

        if tagName in IMPORTANT_TAGS:
            self.is_in[tagName] = False

    # Handle text data
    def characters(self, chars):
        if self.is_in["DescriptorName"] and self.is_in["String"]:
            if "/".join(self.path) == "DescriptorRecordSet/DescriptorRecord/DescriptorName/String":
                self.title += chars
        if self.is_in["Term"] and self.is_in["String"]:
            self.terms.add(chars)
            if self.RecordPreferredTermYN == "Y":
                self.generic_names.add(chars)
        if self.is_in["DescriptorUI"]:
            self.id = chars
        if self.is_in["TreeNumber"]:
            self.tree_numbers.add(chars)

    # Handle startDocument
    def startDocument(self):
        print('About to start!')

    # Handle endDocument
    def endDocument(self):
        print('Finishing up!')


with open("diseases_dictionary_mesh.csv", "w", encoding="utf-8") as fo:
    writer = csv.writer(fo)

    handler = CustomContentHandler(writer)
    xml.sax.parse(mesh_xml_file_name, handler)


Downloaded MeSH XML dump from https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/desc2024.xml.
About to start!
D000006 Abdomen, Acute {'C23.888.592.612.054.200', 'C23.888.821.030.249'} {'Abdomens, Acute', 'Acute Abdomen', 'Abdomen, Acute', 'Acute Abdomens'}
D000007 Abdominal Injuries {'C26.017'} {'Abdominal Injuries', 'Abdominal Injury', 'Injuries, Abdominal', 'Injury, Abdominal'}
D000008 Abdominal Neoplasms {'C04.588.033'} {'Neoplasm, Abdominal', 'Neoplasms, Abdominal', 'Abdominal Neoplasm', 'Abdominal Neoplasms'}
D000012 Abetalipoproteinemia {'C18.452.648.398.500.440.500', 'C18.452.584.563.500.440.500', 'C16.320.565.398.500.440.500', 'C18.452.584.500.875.440.500'} {'Abetalipoproteinemia', 'Deficiency Diseases, Betalipoprotein', 'Diseases, Betalipoprotein Deficiency', 'Microsomal Triglyceride Transfer Protein Deficiency', 'Disease, Betalipoprotein Deficiency', 'Bassen Kornzweig Disease', 'Deficiency Disease, Betalipoprotein', 'Bassen-Kornzweig Syndrome', 'Betalipoprotein Def

D017116 Back Pain {'C23.888.592.612.107'} {'Back Ache', 'Ache, Back', 'Back Pain with Radiation', 'Vertebrogenic Pain Syndromes', 'Backache', 'Backaches', 'Aches, Back', 'Back Pain without Radiation', 'Pain Syndromes, Vertebrogenic', 'Back Pains', 'Pain Syndrome, Vertebrogenic', 'Pain, Back', 'Vertebrogenic Pain Syndrome', 'Back Aches', 'Back Pain', 'Pains, Back', 'Syndromes, Vertebrogenic Pain', 'Syndrome, Vertebrogenic Pain'}
D001423 Bacterial Infections and Mycoses {'C01.150'} {'Bacterial Infections and Mycoses'}
D001424 Bacterial Infections {'C01.150.252'} {'Bacterial Disease', 'Infection, Bacterial', 'Bacterial Infections', 'Infections, Bacterial', 'Bacterial Diseases', 'Bacterial Infection'}
D001437 Bacteriuria {'C01.915.219', 'C12.950.892.219', 'C12.050.351.968.892.219', 'C12.200.777.892.219'} {'Bacteriurias', 'Bacteriuria'}
D001442 Bacteroides Infections {'C01.150.252.400.110.109'} {'Bacteroides Infections', 'Infection, Bacteroides', 'Infections, Bacteroides', 'Bacteroides Infe

D002549 Diffuse Cerebral Sclerosis of Schilder {'C10.314.350.112', 'C10.114.375.112', 'C10.228.140.695.562.112', 'C10.228.140.400', 'C20.111.258.250.175'} {'Diffuse Cerebral Sclerosis of Schilder', 'Sclerosis, Myelinoclastic Diffuse', 'Alper Disease', "Alpers' Syndrome", "Balo's Concentric Sclerosis", 'Alpers Disease', 'Myelinoclastic Diffuse Sclerosis', 'Alpers Diffuse Degeneration of Cerebral Gray Matter with Hepatic Cirrhosis', "Alper's Disease", 'Diffuse Scleroses, Myelinoclastic', 'Scleroses, Myelinoclastic Diffuse', "Concentric Sclerosis, Balo's", 'Syndrome, Alpers-Huttenlocher', 'Myelinoclastic Diffuse Scleroses', "Alper's Syndrome", 'Alpers Syndrome', 'Syndrome, Alpers', 'Diffuse Sclerosis, Myelinoclastic', "Disease, Schilder's", 'Cerebral Sclerosis, Diffuse', 'Sclerosis, Diffuse Cerebral', 'Schilder Disease', 'Poliodystrophia Cerebri', 'Encephalitis Periaxialis Concentrica', 'Diffuse Cerebral Sclerosis', 'Sudanophilic Cerebral Sclerosis', 'Progressive Sclerosing Poliodystrophi

D003872 Dermatitis {'C17.800.174'} {'Dermatitis', 'Dermatitides'}
D003873 Dermatitis, Exfoliative {'C17.800.174.318', 'C17.800.815.318'} {'Dermatitis Exfoliative Generalised', 'Exfoliative Generalized, Dermatitis', 'Exfoliative Generalised, Dermatitis', 'Dermatitis Exfoliative Generaliseds', 'Dermatitis, Exfoliative', 'Dermatitis Exfoliative Generalized', 'Dermatitis Exfoliativa', 'Erythroderma', 'Erythrodermas', 'Exfoliative, Dermatitis', 'Exfoliative Dermatitides', 'Dermatitis Exfoliative', 'Dermatitis Exfoliatives', 'Exfoliative Dermatitis', 'Exfoliatives, Dermatitis', 'Generalised, Dermatitis Exfoliative'}
D003874 Dermatitis Herpetiformis {'C20.111.318', 'C17.800.174.360', 'C17.800.865.360'} {'Duhring Disease', "Disease, Duhring's", 'Duhrings Disease', 'Disease, Duhring', "Duhring's Disease", 'Dermatitis Herpetiformis'}
D003875 Drug Eruptions {'C25.100.468.380', 'C17.800.174.600'} {'Drug Eruption, Maculopapular', 'Morbilliform Drug Reaction', 'Dermatitis Medicamentosa', 'Reaction, 

D004891 Erythema Induratum {'C17.300.710.329', 'C17.800.838.765.820.320', 'C01.150.252.819.820.260', 'C01.800.720.820.260', 'C17.800.566.329', 'C17.800.229.329'} {'Bazin Disease', 'Bazins Disease', 'Tuberculosis Cutis Indurativa', 'Erythema Induratum', "Bazin's Disease", 'Disease, Bazin', "Disease, Bazin's"}
D013262 Erythema Multiforme {'C17.800.865.475', 'C17.800.229.400'} {'Erythema Multiforme'}
D004893 Erythema Nodosum {'C17.800.174.600.375', 'C17.800.229.413', 'C25.100.468.380.375'} {'Erythema Nodosum'}
D004894 Erythrasma {'C01.150.252.410.040.246.694', 'C01.800.720.320', 'C17.800.838.765.320', 'C01.150.252.819.320'} {'Erythrasma'}
D007567 Erythroblastosis, Fetal {'C20.306', 'C16.614.304', 'C16.300.060', 'C15.378.295', 'C12.050.703.277.060'} {'Newborn Hemolytic Diseases', 'Fetalis, Erythroblastosis', 'Fetali, Erythroblastosis', 'Newborn Hemolytic Disease', 'Erythroblastosis Fetalis', 'Fetal Erythroblastosis', 'Erythroblastoses, Fetal', 'Fetal Erythroblastoses', 'Erythroblastosis Fe

...

D054222 Cobblestone Lissencephaly {'C16.131.666.507.450.499.249', 'C10.500.507.450.499.249', 'C16.131.666.507.450.249'} {'Dysplasia, Cobblestone', 'Dysplasias, Cobblestone', 'Cobblestone Dysplasias', 'Cobblestone Complex', 'Cobblestone Dysplasia', 'Lissencephalies, Cobblestone', 'Lissencephaly Type 2', 'Cobblestone Lissencephalies', 'Cobblestone Lissencephaly', 'Lissencephaly, Cobblestone'}
D054243 Vesicular Stomatitis {'C01.925.782.580.830.825', 'C07.465.864.968', 'C22.952'} {'Vesicular Stomatitis', 'Stomatitides, Vesicular', 'Vesicular Stomatitides', 'Stomatitis, Vesicular'}
D054312 Buruli Ulcer {'C17.800.893.295', 'C01.150.252.410.040.552.475.247'} {'Buruli Ulcer Disease', 'Ulcer Disease, Buruli', 'Mycobacterium ulcerans Infections', 'Buruli Ulcer Diseases', 'Infection, Mycobacterium ulcerans', 'Mycobacterium ulcerans Infection', 'Ulcer Diseases, Buruli', 'Disease, Buruli Ulcer', 'Infections, Mycobacterium ulcerans', 'Ulcer, Buruli', 'Diseases, Buruli Ulcer', 'Buruli Ulcer'}
D023903

D058968 Pythiosis {'C22.761', 'C01.610.701.844'} {'Infection, Pythium insidiosum', 'Pythium insidiosum Infections', 'Pythium insidiosum Infection', 'Pythioses', 'Pythiosis Insidiosi', 'Pythiosis'}
D059041 Plagiocephaly {'C16.131.621.207.707', 'C05.660.207.707'} {'Plagiocephalies', 'Plagiocephaly'}
D059226 Nociceptive Pain {'C23.888.592.612.720'} {'Nociceptive Pains', 'Somatic Pain', 'Pain, Nociceptive', 'Pain, Tissue', 'Tissue Pain', 'Tissue Pains', 'Pains, Nociceptive', 'Pain, Somatic', 'Pains, Somatic', 'Somatic Pains', 'Nociceptive Pain', 'Pains, Tissue'}
D013478 Renal Nutcracker Syndrome {'C12.200.777.419.787', 'C12.050.351.968.419.787', 'C12.950.419.787'} {'Phenomenons, Renal Nutcracker', 'Renal Nutcracker Syndrome', 'Nutcracker Syndromes, Renal', 'Nutcracker Phenomenon, Renal', 'Phenomenon, Renal Nutcracker', 'Nutcracker Phenomenons, Renal', 'Renal Nutcracker Phenomenons', 'Renal Vein Entrapment Syndrome', 'Renal Nutcracker Phenomenon', 'Nutcracker Syndrome, Renal', 'Renal Nutcra

D065703 Malformations of Cortical Development, Group I {'C10.500.507.400', 'C16.131.666.507.400'} {'Malformations of Cortical Development, Group I', 'Malformations Due to Abnormal Neuronal and Glial Proliferation or Apoptosis', 'Abnormal Proliferation Cortical Malformations', 'Malformations Secondary to Abnormal Neuronal and Glial Proliferation or Apoptosis', 'Cortical Malformations, Group I'}
D065704 Malformations of Cortical Development, Group III {'C16.131.666.507.500', 'C10.500.507.500'} {'Malformations Secondary to Abnormal Postmigrational Development', 'Cortical Malformations, Group III', 'Malformations of Cortical Development, Group III', 'Malformations Due to Abnormal Postmigrational Development'}
D065705 Hemimegalencephaly {'C16.131.666.507.400.249.500', 'C10.500.507.400.249.500', 'C16.131.621.207.532.500', 'C05.660.207.536.500'} {'Megalencephalies, Unilateral', 'Unilateral Megalencephalies', 'Hemimegalencephaly', 'Unilateral Megalencephaly', 'Macrocephaly, Unilateral', 'Unila

D000072281 Lymphadenopathy {'C15.604.338'} {'Adenopathy', 'Lymphadenopathy', 'Lymphadenopathies', 'Adenopathies'}
D001943 Breast Cancer Lymphedema {'C23.550.767.082', 'C15.604.496.160'} {'Breast Cancer Treatment Related Lymphedema', 'Breast Cancer Related Arm Lymphedema', 'Post-mastectomy Lymphedema', 'Post-mastectomy Lymphedemas', 'Breast Cancer Lymphedema', 'Breast Cancer Related Lymphedema', 'Postmastectomy Lymphedema', 'Lymphedema, Postmastectomy', 'Postmastectomy Lymphedemas', 'Breast Cancer Lymphedemas', 'Lymphedema, Post-mastectomy', 'Breast Cancer-Related Arm Lymphedema', 'Post mastectomy Lymphedema', 'Lymphedema, Breast Cancer', 'Lymphedemas, Postmastectomy', 'Breast Cancer Treatment-Related Lymphedema'}
D000072657 ST Elevation Myocardial Infarction {'C23.550.717.489.750.875', 'C23.550.513.355.750.875', 'C14.280.647.500.875', 'C14.907.585.500.875'} {'ST Elevation Myocardial Infarction', 'STEMI', 'ST Segment Elevation Myocardial Infarction', 'ST Elevated Myocardial Infarction'}

D000088282 Corticobasal Degeneration {'C10.574.945.312'} {'Corticobasal Syndrome', 'Cortico basal Ganglionic Degeneration', 'Cortico-basal Ganglionic Degeneration', 'Cortico basal Degeneration', 'Corticobasal Degeneration', 'Cortico-basal Degeneration'}
D000088442 MINOCA {'C23.550.717.489.750.328', 'C14.907.585.500.422', 'C14.280.647.500.328', 'C23.550.513.355.750.328'} {'MINOCA', 'Myocardial Infarction with Nonobstructive Coronary Arteries', 'Myocardial Infarction and Non obstructed Coronary Arteries', 'Myocardial Infarction with Non-obstructive Coronary Arteries', 'Myocardial Infarction and Non-obstructed Coronary Arteries', 'Myocardial Infarction and Nonobstructed Coronary Arteries', 'Myocardial Infarction with Non obstructive Coronary Arteries'}
D002908 Persistent Infection {'C23.550.291.902', 'C01.645'} {'Persistent Infection', 'Infection, Persistent', 'Chronic Infection', 'Infection, Long Term', 'Long Term Infections', 'Chronic Infections', 'Persistent Infections', 'Infection, Ch